In [1]:
import torch
import torchvision
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils
from torchvision import datasets
from torchvision.utils import save_image

import skimage 
import math
# import io
# import requests
# from PIL import Image

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import sys
import os


# Pytorch tests to understand some ideas

## Test update patches of tensor (replace some values only instead of operation in entire tensor)

In [2]:
t = torch.ones(10,10)

In [3]:
t


    1     1     1     1     1     1     1     1     1     1
    1     1     1     1     1     1     1     1     1     1
    1     1     1     1     1     1     1     1     1     1
    1     1     1     1     1     1     1     1     1     1
    1     1     1     1     1     1     1     1     1     1
    1     1     1     1     1     1     1     1     1     1
    1     1     1     1     1     1     1     1     1     1
    1     1     1     1     1     1     1     1     1     1
    1     1     1     1     1     1     1     1     1     1
    1     1     1     1     1     1     1     1     1     1
[torch.FloatTensor of size (10,10)]

In [4]:
t[3:5, 3:5] = torch.zeros(2,2)

In [5]:
t[3:5, 3:6]


 0  0  1
 0  0  1
[torch.FloatTensor of size (2,3)]

In [6]:
tpatch = t[5:7, 5:7]
tpatch = torch.rand(2,2)
tpatch2 = torch.rand(10,2)

In [7]:
tpatch


 0.5234  0.6426
 0.3535  0.0635
[torch.FloatTensor of size (2,2)]

In [8]:
t[:, 5:7] 


    1     1
    1     1
    1     1
    1     1
    1     1
    1     1
    1     1
    1     1
    1     1
    1     1
[torch.FloatTensor of size (10,2)]

In [18]:
t[:, 5:7]  = tpatch

RuntimeError: The expanded size of the tensor (10) must match the existing size (2) at non-singleton dimension 0

In [10]:
t[:, 5:7]  = tpatch2

In [12]:
t


 1.0000  1.0000  1.0000  1.0000  1.0000  0.8014  0.8230  1.0000  1.0000  1.0000
 1.0000  1.0000  1.0000  1.0000  1.0000  0.0922  0.5681  1.0000  1.0000  1.0000
 1.0000  1.0000  1.0000  1.0000  1.0000  0.7856  0.3420  1.0000  1.0000  1.0000
 1.0000  1.0000  1.0000  0.0000  0.0000  0.5735  0.3144  1.0000  1.0000  1.0000
 1.0000  1.0000  1.0000  0.0000  0.0000  0.2125  0.3810  1.0000  1.0000  1.0000
 1.0000  1.0000  1.0000  1.0000  1.0000  0.6290  0.6791  1.0000  1.0000  1.0000
 1.0000  1.0000  1.0000  1.0000  1.0000  0.5470  0.7892  1.0000  1.0000  1.0000
 1.0000  1.0000  1.0000  1.0000  1.0000  0.2704  0.7376  1.0000  1.0000  1.0000
 1.0000  1.0000  1.0000  1.0000  1.0000  0.6119  0.5020  1.0000  1.0000  1.0000
 1.0000  1.0000  1.0000  1.0000  1.0000  0.7909  0.4424  1.0000  1.0000  1.0000
[torch.FloatTensor of size (10,10)]

In [19]:
torch.equal(tpatch2, t[:, 5:7])

True

## Test transforms applied to tensor

In [13]:
from helper_modules import *

In [14]:

class DownsamplerLayer(nn.Module):
    """
    """
    def __init__(self, width, height):
        super().__init__()
        self.resizer = transforms.Compose([
                        transforms.ToTensor(),
                        # transforms.ToPILImage(),  #is this correct? will this be slow??
                        transforms.Resize(width, height),
                        transforms.ToTensor()
                        ])
    def forward(self, x):
        return self.resizer(x)
        #return self.conv(x)

In [15]:
dl = DownsamplerLayer(5,5)


In [16]:
tr = transforms.Resize(5,5)

In [17]:
tr(t.data.)

SyntaxError: invalid syntax (<ipython-input-17-68fa25079c6e>, line 1)

In [ ]:
dl(t.data)

In [ ]:
import io
import requests
from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

%matplotlib inline

In [ ]:
IMG_URL = 'https://s3.amazonaws.com/outcome-blog/wp-content/uploads/2017/02/25192225/cat.jpg'
response = requests.get(IMG_URL)
img_pil = Image.open(io.BytesIO(response.content))#.convert("RGB")

#test one:
preprocess = transforms.Compose([
    #transforms.Resize(50),
    transforms.ToTensor()
])

to_pil = torchvision.transforms.ToPILImage()



img_tensor = preprocess(img_pil)
img_tensor.shape

In [ ]:
img_tensor[-0.5:0, -0.5:0]

In [ ]:
to_pil(img_tensor)

## Test AdaptiveAvgPool2D as downsampling


In [ ]:
m = nn.AdaptiveAvgPool2d((5,5))

In [ ]:
input = torch.randn(1, 10, 10)


In [ ]:
m(input)

In [ ]:
ds = nn.AdaptiveAvgPool2d((50,75))

In [ ]:
downsampled = ds(img_tensor)
to_pil(downsampled)

In [ ]:
downsampled.shape

## Test upsampling tensor

In [ ]:
us = nn.AdaptiveMaxPool2d((100,150))

In [ ]:
upsampled = us(downsampled)

In [ ]:
to_pil(upsampled)

## Tests with bilinear interpolation

Documentation and examples from [here](https://gist.github.com/peteflorence/a1da2c759ca1ac2b74af9a83f69ce20e)



In [ ]:
import torch.nn.functional
dtype = torch.cuda.FloatTensor
dtype_long = torch.cuda.LongTensor
import time

In [ ]:
def bilinear_interpolate_torch_gridsample(image, samples_x, samples_y):
                                                # input image is: W x H x C
    image = image.permute(2,0,1)                # change to:      C x W x H
    image = image.unsqueeze(0)                  # change to:  1 x C x W x H
    samples_x = samples_x.unsqueeze(2)
    samples_x = samples_x.unsqueeze(3)
    samples_y = samples_y.unsqueeze(2)
    samples_y = samples_y.unsqueeze(3)
    samples = torch.cat([samples_x, samples_y],3)
    samples[:,:,:,0] = (samples[:,:,:,0]/(W-1)) # normalize to between  0 and 1
    samples[:,:,:,1] = (samples[:,:,:,1]/(H-1)) # normalize to between  0 and 1
    samples = samples*2-1                       # normalize to between -1 and 1
    return torch.nn.functional.grid_sample(image, samples)

In [ ]:
# Correctness test
W, H, C = 5, 5, 1
test_image = torch.ones(W,H,C).type(dtype)
test_image[3,3,:] = 4
test_image[3,4,:] = 3

test_samples_x = torch.FloatTensor([[3.2]]).type(dtype)
test_samples_y = torch.FloatTensor([[3.4]]).type(dtype)

In [ ]:
img = bilinear_interpolate_torch_gridsample(test_image, test_samples_x, test_samples_y)

# Benchmark
# start = time.time()
# bilinear_interpolate_torch_gridsample(image, samples_x, samples_y)
# print ("torch gridsample took ", time.time() - start)

In [ ]:
test_samples_x

In [ ]:
test_image

# other tests

In [23]:
v = Variable(torch.FloatTensor([1.0/3]))

In [24]:
t = Variable(torch.rand(3,3))

In [25]:
t * v


 0.2249  0.2423  0.1418
 0.2403  0.0933  0.0808
 0.0480  0.0302  0.0269
[torch.FloatTensor of size (3,3)]

In [26]:
t


 0.6748  0.7268  0.4255
 0.7208  0.2798  0.2424
 0.1441  0.0905  0.0807
[torch.FloatTensor of size (3,3)]

In [28]:
v.requires_grad

False

In [30]:
v.is_leaf

True